In [1]:
import pickle
import re
import pandas as pd

# mapping ctv3 to icd10
ctv_icd_map = pd.read_excel("../../../data/all_lkps_maps_v3.xlsx", 
                            engine='openpyxl', sheet_name = ["read_ctv3_icd10"])["read_ctv3_icd10"]
ctv_icd_map = ctv_icd_map[~ctv_icd_map["mapping_status"].isin(["R","U"])]
ctv_icd_map = ctv_icd_map[ctv_icd_map["refine_flag"]!="M"]
ctv_icd_map = ctv_icd_map[ctv_icd_map["element_num"]==0] 
ctv_icd_map["icd10_code"] = ctv_icd_map["icd10_code"].apply(lambda x: re.sub('[a-zA-Z]+$', '', x))
read2_read3 = pd.read_excel("../../../data/all_lkps_maps_v3.xlsx", 
                            engine='openpyxl', sheet_name = ["read_v2_read_ctv3"])["read_v2_read_ctv3"]
read2_read3


In [3]:
t=pd.read_csv("../../../data/gp_clinical.txt", delimiter='\t', chunksize=10000)#Your own file
next(t)

,eid,data_provider,event_dt,read_2,read_3,value1,value2,value3
0,1000016,4,01/06/2006,68P..,NaN,NaN,NaN,NaN
1,1000016,4,12/10/2015,424..,NaN,NaN,NaN,NaN
2,1000016,4,19/11/2007,4K22.,NaN,NaN,NaN,NaN
3,1000016,4,21/09/2015,1M10.,NaN,NaN,NaN,NaN
4,1000016,4,04/10/2016,73131,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
9995,1000399,3,09/01/2013,NaN,XE2q0,139.000,NaN,NaN
9996,1000399,3,19/11/2007,NaN,XE2q9,1.520,NaN,NaN
9997,1000399,3,24/01/1992,NaN,2469.,150.000,NaN,NaN
9998,1000399,3,10/11/2014,NaN,2469.,122.000,NaN,NaN


In [4]:
# adjusting icd10
phe_icd = pd.read_csv("../../../data/phecode_icd10.csv")
phe_icd_code = phe_icd.ALT_CODE.tolist()

In [5]:
for i,code in enumerate(phe_icd_code):
    if code[-1]=='X':
        phe_icd_code[i]=code[:-1]
        print(code,code[:-1])

A33X A33
A34X A34
A35X A35
A38X A38
A46X A46
A55X A55
A57X A57
A58X A58
A64X A64
A65X A65
A70X A70
A78X A78
A86X A86
A89X A89
A94X A94
A99X A99
B03X B03
B04X B04
B07X B07
B09X B09
B24X B24
B49X B49
B54X B54
B59X B59
B64X B64
B72X B72
B73X B73
B75X B75
B79X B79
B80X B80
B86X B86
B89X B89
B91X B91
B92X B92
B99X B99
C01X C01
C07X C07
C12X C12
C19X C19
C20X C20
C23X C23
C33X C33
C37X C37
C52X C52
C55X C55
C56X C56
C58X C58
C61X C61
C64X C64
C65X C65
C66X C66
C73X C73
C97X C97
D24X D24
D27X D27
D34X D34
D45X D45
D62X D62
D65X D65
D66X D66
D67X D67
D70X D70
D71X D71
D77X D77
E02X E02
E15X E15
E40X E40
E41X E41
E42X E42
E43X E43
E43X E43
E43X E43
E45X E45
E46X E46
E52X E52
E54X E54
E58X E58
E59X E59
E60X E60
E65X E65
E68X E68
E86X E86
F03X F03
F09X F09
F21X F21
F24X F24
F29X F29
F39X F39
F54X F54
F55X F55
F59X F59
F61X F61
F69X F69
F82X F82
F88X F88
F89X F89
F99X F99
G01X G01
G07X G07
G08X G08
G09X G09
G10X G10
G20X G20
G26X G26
G35X G35
G64X G64
G92X G92
G98X G98
H46X H46
H55X H55
H71X H71
H

In [6]:
for i in range(len(ctv_icd_map)):
    if len(ctv_icd_map.iloc[i,1])>=4 and ctv_icd_map.iloc[i,1] not in phe_icd_code:
        ctv_icd_map.iloc[i,1] = ctv_icd_map.iloc[i,1][:-1] 
        

# two-step mapping
data_chunks = []
reader = pd.read_csv("../../../data/gp_clinical.txt", delimiter='\t', chunksize=10000)
for id,chunk in enumerate(reader):
    read2_part = chunk.loc[~chunk['event_dt'].isna() & ~chunk['read_2'].isna()]
    read3_part = chunk.loc[~chunk['event_dt'].isna() & ~chunk['read_3'].isna()]
    merge_chunk2 = read2_part[["eid","read_2",'event_dt']].merge(read2_read3[['READV2_CODE', 'READV3_CODE']], left_on='read_2', right_on='READV2_CODE')
    merge_chunk2 = merge_chunk2.merge(ctv_icd_map, left_on='READV3_CODE', right_on='read_code')
    merge_chunk3 = read3_part.merge(ctv_icd_map, left_on="read_3", right_on="read_code")
    data_chunks.append(merge_chunk2[['eid', 'icd10_code','event_dt']])
    data_chunks.append(merge_chunk3[['eid', 'icd10_code','event_dt']])
readv3_icd = pd.concat(data_chunks, axis=0)
readv3_icd.drop_duplicates(inplace=True)
print("The length of readv3_icd %d"%(len(readv3_icd)))

The length of readv3_icd 12521759


In [8]:
df_icd = readv3_icd.groupby('eid')[['icd10_code','event_dt']].agg(list)

In [ ]:
def format_yyyymmdd(date):
    date=date.split('/')
    return f'{date[2]}-{date[1]}-{date[0]}'

In [10]:
tpp_icd_dict = {}
for i in df_icd.index:
    temp={}
    for s,code in enumerate(df_icd['icd10_code'][i]):
        t=format_yyyymmdd(df_icd['event_dt'][i][s])
        if code in temp.keys():
            if t<temp[code]:
                temp[code]=t
        else:
            temp[code]=t
    tpp_icd_dict[i]=temp
pickle.dump(tpp_icd_dict, open('../../../results/cache/pricare', 'wb'))

In [11]:
tpp_icd_dict = {}
for i in df_icd.index:
    temp={}
    for s,code in enumerate(df_icd['icd10_code'][i]):
        t=format_yyyymmdd(df_icd['event_dt'][i][s])
        if code in temp.keys():
            temp[code].append(t)
        else:
            temp[code]=[t]
    tpp_icd_dict[i]=temp
pickle.dump(tpp_icd_dict, open('../../../results/cache/pricarer2', 'wb'))